In [1]:
!pip install -qU agno duckduckgo-search yfinance groq gradio

In [2]:
import gradio as gr
import os
import json
import time
from agno.agent import Agent
from agno.models.groq import Groq
from agno.tools.duckduckgo import DuckDuckGoTools
from google.colab import userdata

In [3]:
output_dir = "./travel_output"
os.makedirs(output_dir, exist_ok=True)

groq_api_key = userdata.get('GROQ_API_KEY')
basic_llm = Groq(id="qwen-2.5-32b", api_key=groq_api_key)

In [4]:

# Travel Researcher Agent:
research_agent = Agent(
    name="Travel Researcher",
    role="Find travel destinations and attractions",
    model=basic_llm,
    tools=[DuckDuckGoTools()],
    instructions=(
        "Provide the latest travel information with verified sources. "
        "Return your results in a clear, step-by-step format with headings for each section."
    ),
    markdown=True
)


# Itinerary Builder Agent:
planner_agent = Agent(
    name="Itinerary Builder",
    role="Create optimized travel plans",
    model=basic_llm,
    instructions=(
        "Include transportation, accommodation, and activity recommendations. "
        "Format the itinerary as a numbered list with clear steps for each day."
    ),
    markdown=True
)



# Budget Analyst Agent:
budget_agent = Agent(
    name="Budget Analyst",
    role="Plan and optimize travel budgets",
    model=basic_llm,
    instructions=(
        "Analyze the trip costs, provide a breakdown of expected expenses, and include cost-saving recommendations. "
        "Return your analysis in clear, organized steps with headings for each expense category."
    ),
    markdown=True
)



# Combine the individual agents into a team agent:
travel_team = Agent(
    team=[research_agent, planner_agent, budget_agent],
    model=basic_llm,
    instructions=[
        "Consider budget and time constraints, safety recommendations, and local cuisine suggestions.",
        "Return a well-organized, step-by-step travel plan that integrates destination research, itinerary planning, and budget analysis.",
        "Each section should have a clear heading and corresponding details."
    ],
    markdown=True
)


#### Gradio UI

In [5]:
def plan_travel(user_query: str) -> str:
    """
    Runs the travel team agent on the user query and returns the organized travel plan.
    """
    result = travel_team.run(user_query)

    return str(result)

with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("# 🌍 AI Travel Planner")

    query = gr.Textbox(
        label="Enter your travel request",
        placeholder="e.g., 10-day family trip to Japan under $5000",
        lines=2
    )
    btn = gr.Button("Generate Plan", variant="primary")
    output = gr.Markdown()

    btn.click(fn=plan_travel, inputs=query, outputs=output)

app.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://181b8b18cd094ccadb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
